In [1]:
# ============================================================================
# CELL 1: Install Dependencies (Xử lý triệt để conflicts trên Kaggle)
# ============================================================================
import subprocess
import sys

print("="*60)
print("CÀI ĐẶT DEPENDENCIES")
print("="*60)

# Bước 1: Uninstall các packages có thể conflict
print("Đang uninstall packages cũ...")
packages_to_uninstall = ["transformers", "peft", "accelerate", "datasets"]
for pkg in packages_to_uninstall:
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "-q", pkg], 
                              stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL)
    except:
        pass

# Bước 2: Cài transformers version mới nhất
print("Đang cài transformers...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "--upgrade", 
                       "transformers"], stderr=subprocess.DEVNULL)

# Bước 3: Cài các dependencies của transformers
print("Đang cài dependencies của transformers...")
transformers_deps = [
    "packaging>=20.0", "pyyaml>=5.1", "regex!=2019.12.17", "requests",
    "tokenizers>=0.19.1", "safetensors>=0.4.1", "tqdm>=4.27", "numpy",
    "huggingface-hub>=0.20.0",
]
for dep in transformers_deps:
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", dep], 
                              stderr=subprocess.DEVNULL)
    except:
        pass

# Bước 4: Cài peft, accelerate, datasets
print("Đang cài peft, accelerate, datasets...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "peft>=0.7.0"], 
                      stderr=subprocess.DEVNULL)
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "accelerate"], 
                      stderr=subprocess.DEVNULL)
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "datasets"], 
                      stderr=subprocess.DEVNULL)

# Bước 5: Cài các packages khác
print("Đang cài các packages khác...")
other_packages = ["sentencepiece", "nltk", "rouge-score", 
                  "portalocker", "colorama", "sacrebleu", "sacremoses",
                  "soundfile", "librosa", "av", "yt-dlp", "scipy"]
for pkg in other_packages:
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg], 
                              stderr=subprocess.DEVNULL)
    except:
        pass

print("✓ Dependencies đã được cài đặt")


CÀI ĐẶT DEPENDENCIES
Đang uninstall packages cũ...
Đang cài transformers...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.3 MB/s eta 0:00:00
Đang cài dependencies của transformers...
Đang cài peft, accelerate, datasets...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [2]:
# ============================================================================
# CELL 2: Import Libraries (với patch để fix lỗi transformers)
# ============================================================================
import os
import json
import torch
import warnings
import pandas as pd
from tqdm import tqdm

# Disable flash attention
os.environ["DISABLE_FLASH_ATTENTION"] = "1"

# QUAN TRỌNG: Patch các hàm utils TRƯỚC KHI import transformers
print("Đang patch các hàm cần thiết trước khi import transformers...")

# Hàm helper để patch TẤT CẢ các hàm utils có thể thiếu
def patch_all_transformers_utils():
    """Patch tất cả các hàm utils có thể thiếu vào transformers.utils"""
    try:
        import transformers
        if not hasattr(transformers, 'utils'):
            from types import ModuleType
            transformers.utils = ModuleType('transformers.utils')
        
        utils_module = transformers.utils
        
        # Patch TẤT CẢ các hàm utils có thể thiếu
        all_utils_map = {
            # Audio utils
            'is_soundfile_available': 'soundfile',
            'is_librosa_available': 'librosa',
            'is_torchcodec_available': 'torchcodec',
            'is_torchaudio_available': 'torchaudio',
            'is_scipy_available': 'scipy',
            # Video utils
            'is_av_available': 'av',
            'is_yt_dlp_available': 'yt_dlp',
            # Image utils
            'is_pillow_available': 'PIL',
            'is_vision_available': 'torchvision',
            # Other utils
            'is_tensorflow_available': 'tensorflow',
            'is_flax_available': 'jax',
            'is_safetensors_available': 'safetensors',
        }
        
        for func_name, module_name in all_utils_map.items():
            if not hasattr(utils_module, func_name):
                try:
                    __import__(module_name)
                    setattr(utils_module, func_name, lambda: True)
                except (ImportError, ModuleNotFoundError):
                    setattr(utils_module, func_name, lambda: False)
        
        # Patch các constants có thể thiếu
        constants_to_patch = {
            'AUDIO_TOKENIZER_NAME': 'audio_tokenizer',
            'IMAGE_TOKENIZER_NAME': 'image_tokenizer',
            'VIDEO_TOKENIZER_NAME': 'video_tokenizer',
        }
        
        for const_name, default_value in constants_to_patch.items():
            if not hasattr(utils_module, const_name):
                setattr(utils_module, const_name, default_value)
        
        return True
    except Exception as e:
        print(f"⚠ Lỗi khi patch utils: {e}")
        # Patch cơ bản nếu có lỗi
        try:
            import transformers
            if not hasattr(transformers, 'utils'):
                from types import ModuleType
                transformers.utils = ModuleType('transformers.utils')
            utils_module = transformers.utils
            # Patch các hàm và constants quan trọng nhất
            critical_items = {
                'is_torchcodec_available': lambda: False,
                'is_yt_dlp_available': lambda: False,
                'is_soundfile_available': lambda: False,
                'is_av_available': lambda: False,
                'AUDIO_TOKENIZER_NAME': 'audio_tokenizer',
                'IMAGE_TOKENIZER_NAME': 'image_tokenizer',
                'VIDEO_TOKENIZER_NAME': 'video_tokenizer',
            }
            for name, value in critical_items.items():
                if not hasattr(utils_module, name):
                    setattr(utils_module, name, value)
        except:
            pass
        return False

# Thử patch ngay nếu transformers đã được import
try:
    import transformers
    if patch_all_transformers_utils():
        print("✓ Đã patch các hàm utils")
except:
    pass

# Import với error handling và patch
try:
    # Import transformers từng bước để có thể patch
    import transformers
    
    # Patch TẤT CẢ utils ngay sau khi import transformers
    if patch_all_transformers_utils():
        print("✓ Đã patch các hàm utils")
    
    # Patch các hàm khác
    try:
        import transformers.utils.import_utils as import_utils_module
        if not hasattr(import_utils_module, 'requires'):
            def requires(*args, **kwargs):
                def decorator(func):
                    return func
                return decorator
            import_utils_module.requires = requires
    except:
        pass
    
    try:
        import transformers.pytorch_utils as pytorch_utils_module
        if not hasattr(pytorch_utils_module, 'is_torch_greater_or_equal_than_2_3'):
            def is_torch_greater_or_equal_than_2_3():
                version = torch.__version__.split('.')
                if len(version) >= 2:
                    major, minor = int(version[0]), int(version[1])
                    return (major > 2) or (major == 2 and minor >= 3)
                return False
            pytorch_utils_module.is_torch_greater_or_equal_than_2_3 = is_torch_greater_or_equal_than_2_3
    except:
        pass
    
    # Import các class cần thiết
    from transformers import (
        AutoModelForSeq2SeqLM,
        AutoTokenizer,
        TrainingArguments,
        Trainer,
        DataCollatorForSeq2Seq
    )
    from peft import LoraConfig, get_peft_model, TaskType
    from datasets import Dataset, DatasetDict
    import sacrebleu
    print("✓ Đã import các thư viện thành công")
    
except Exception as e:
    print(f"⚠ Lỗi import: {e}")
    print("Đang thử cài lại transformers...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--force-reinstall", 
                           "--upgrade", "-q", "transformers"], stderr=subprocess.DEVNULL)
    import transformers
    patch_all_transformers_utils()
    from transformers import (
        AutoModelForSeq2SeqLM,
        AutoTokenizer,
        TrainingArguments,
        Trainer,
        DataCollatorForSeq2Seq
    )
    from peft import LoraConfig, get_peft_model, TaskType
    from datasets import Dataset, DatasetDict
    import sacrebleu
    print("✓ Đã import các thư viện thành công (sau khi cài lại)")

# Suppress warnings
warnings.filterwarnings("ignore")

Đang patch các hàm cần thiết trước khi import transformers...


2025-12-17 02:23:38.856396: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765938219.049031      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765938219.106278      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

✓ Đã patch các hàm utils
✓ Đã patch các hàm utils
✓ Đã import các thư viện thành công


In [3]:
# ============================================================================
# CELL 3: Configuration (Có thể chỉnh sửa)
# ============================================================================
class Config:
    # Data paths (điều chỉnh theo cấu trúc Kaggle)
    DATA_DIR = "/kaggle/input/medicaldataset-vlsp/MedicalDataset_VLSP"
    
    # Model - Chọn model ổn định, dễ chạy trên Kaggle
    # KHUYẾN NGHỊ: Dùng Helsinki-NLP vì đơn giản, ổn định, chuyên về translation
    # Hoặc mT5-small nếu muốn model lớn hơn một chút
    
    MODEL_NAME = "Helsinki-NLP/opus-mt-vi-en"  # Helsinki - nhỏ, ổn định, chuyên translation
    # MODEL_NAME = "google/mt5-small"  # mT5 small - nếu muốn thử mT5
    # MODEL_NAME = "google/mt5-base"  # mT5 base - nếu có đủ memory
    
    USE_LORA = True
    LORA_R = 16
    LORA_ALPHA = 32
    LORA_DROPOUT = 0.1
    
    # Model type - Tự động detect hoặc set thủ công
    # "nllb", "mt5", "helsinki", "qwen", "auto"
    MODEL_TYPE = "auto"  # Tự động detect từ MODEL_NAME
    
    # Optimization - Tối ưu cho P100/T4X2
    USE_UNSLOTH = False  # Tạm tắt do lỗi compatibility
    
    # Training - TỐI ƯU CHO KAGGLE (nhanh, tiết kiệm thời gian)
    OUTPUT_DIR = "/kaggle/working/checkpoints"
    NUM_EPOCHS = 1  # GIẢM XUỐNG 1 EPOCH để chạy nhanh hơn (có thể tăng sau)
    BATCH_SIZE = 8  # TĂNG batch size lên 8 (nếu memory đủ, sẽ nhanh hơn)
    GRADIENT_ACCUMULATION_STEPS = 4  # Giảm xuống 4 (effective batch = 32, vẫn ổn)
    LEARNING_RATE = 5e-5
    MAX_LENGTH = 128  # GIẢM xuống 128 để nhanh hơn (đủ cho hầu hết câu)
    GRADIENT_CHECKPOINTING = False  # TẠM TẮT để tránh xung đột với LoRA
    
    # Data - GIẢM SỐ SAMPLES để chạy nhanh hơn
    MAX_SAMPLES = 500000  # GIẢM xuống 30,000 samples (đủ để train, nhanh hơn nhiều)
    
    # Tối ưu thời gian - Giảm tần suất logging/eval/save
    LOGGING_STEPS = 100  # Log ít hơn (từ 50 lên 100)
    EVAL_STEPS = 500  # Eval ít hơn (từ 250 lên 500)
    SAVE_STEPS = 1000  # Save ít hơn (từ 500 lên 1000)
    EVAL_STRATEGY = "no"  # TẮT evaluation trong training để tiết kiệm thời gian (có thể đổi thành "steps" nếu muốn)
    
    # Direction
    DIRECTION = "en2vi"  # "en2vi" hoặc "vi2en"
    
    # Evaluation
    PREDICTIONS_FILE = "/kaggle/working/predictions.txt"
    RESULTS_FILE = "/kaggle/working/evaluation_results.json"

config = Config()

In [4]:
# ============================================================================
# CELL 4: Helper Functions (Data Loading)
# ============================================================================
def clean_text(text):
    """Làm sạch văn bản"""
    import re
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    return text

def load_parallel_data(en_file, vi_file, max_samples=None):
    """Đọc dữ liệu song ngữ"""
    pairs = []
    with open(en_file, 'r', encoding='utf-8') as f_en, \
         open(vi_file, 'r', encoding='utf-8') as f_vi:
        en_lines = f_en.readlines()
        vi_lines = f_vi.readlines()
        
        for en_line, vi_line in zip(en_lines, vi_lines):
            en_text = clean_text(en_line)
            vi_text = clean_text(vi_line)
            if en_text and vi_text:
                pairs.append((en_text, vi_text))
                if max_samples and len(pairs) >= max_samples:
                    break
    return pairs

def create_translation_prompt(source_text, source_lang="en", target_lang="vi", model_type="auto"):
    """
    Tạo prompt/input cho translation tùy theo loại model
    
    Args:
        source_text: Văn bản nguồn
        source_lang: Ngôn ngữ nguồn
        target_lang: Ngôn ngữ đích
        model_type: Loại model ("nllb", "mt5", "helsinki", "qwen", "auto")
    """
    # NLLB và Helsinki-NLP không cần prompt, chỉ cần text
    if model_type in ["nllb", "helsinki"]:
        return source_text
    
    # mT5 cần prefix
    elif model_type == "mt5":
        lang_map = {"en": "English", "vi": "Vietnamese"}
        return f"translate {lang_map[source_lang]} to {lang_map[target_lang]}: {source_text}"
    
    # Qwen và các model khác cần prompt đầy đủ
    else:
        lang_map = {"en": "English", "vi": "Vietnamese"}
        return f"Translate the following {lang_map[source_lang]} text to {lang_map[target_lang]}:\n\n{source_text}"


In [5]:
# ============================================================================
# CELL 5: Load and Prepare Data
# ============================================================================
print("Đang tải dữ liệu...")

# Xác định file paths
if config.DIRECTION == "vi2en":
    train_source = os.path.join(config.DATA_DIR, "train.en.txt")
    train_target = os.path.join(config.DATA_DIR, "train.vi.txt")
    test_source = os.path.join(config.DATA_DIR, "public_test.en.txt")
    test_target = os.path.join(config.DATA_DIR, "public_test.vi.txt")
    source_lang, target_lang = "en", "vi"
else:
    train_source = os.path.join(config.DATA_DIR, "train.vi.txt")
    train_target = os.path.join(config.DATA_DIR, "train.en.txt")
    test_source = os.path.join(config.DATA_DIR, "public_test.vi.txt")
    test_target = os.path.join(config.DATA_DIR, "public_test.en.txt")
    source_lang, target_lang = "vi", "en"

# Load data
train_pairs = load_parallel_data(train_source, train_target, max_samples=config.MAX_SAMPLES)
test_pairs = load_parallel_data(test_source, test_target)  # Test data: dùng tất cả

print(f"Train: {len(train_pairs)} pairs")
print(f"Test: {len(test_pairs)} pairs")

# Chia train/val
train_size = int(0.9 * len(train_pairs))
train_data = train_pairs[:train_size]
val_data = train_pairs[train_size:]

# Tạo DataFrames
train_df = pd.DataFrame(train_data, columns=["source", "target"])
val_df = pd.DataFrame(val_data, columns=["source", "target"])
test_df = pd.DataFrame(test_pairs, columns=["source", "target"])

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")


Đang tải dữ liệu...
Train: 500000 pairs
Test: 3000 pairs
Train: 450000, Val: 50000, Test: 3000


In [6]:
# ============================================================================
# CELL 6: Load Model and Tokenizer
# ============================================================================
# Suppress pydantic warnings (không ảnh hưởng đến chức năng)
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")

print("Đang tải mô hình...")

# Detect model type từ MODEL_NAME
def detect_model_type(model_name):
    """Tự động detect loại model từ tên"""
    model_name_lower = model_name.lower()
    if "nllb" in model_name_lower:
        return "nllb"
    elif "mt5" in model_name_lower or ("t5" in model_name_lower and "mt" in model_name_lower):
        return "mt5"
    elif "helsinki" in model_name_lower or "opus" in model_name_lower:
        return "helsinki"
    elif "qwen" in model_name_lower:
        return "qwen"
    else:
        return "auto"

if config.MODEL_TYPE == "auto":
    config.MODEL_TYPE = detect_model_type(config.MODEL_NAME)
    print(f"✓ Detected model type: {config.MODEL_TYPE}")

# Sử dụng Unsloth nếu được bật (tối ưu hóa training)
# Lưu ý: Unsloth chỉ hỗ trợ một số model (Qwen, Llama, Mistral)
if config.USE_UNSLOTH and config.MODEL_TYPE in ["qwen"]:
    try:
        from unsloth import FastLanguageModel
        print("Đang sử dụng Unsloth để tối ưu hóa...")
        
        # Load model với Unsloth (tự động tối ưu cho P100)
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name=config.MODEL_NAME,
            max_seq_length=config.MAX_LENGTH,
            dtype=torch.float16,
            load_in_4bit=False,  # P100 không hỗ trợ 4-bit, dùng float16
            load_in_8bit=False,  # Không dùng 8-bit cho P100
        )
        
        # Áp dụng LoRA với Unsloth - Tối ưu cho P100
        if config.USE_LORA:
            model = FastLanguageModel.get_peft_model(
                model,
                r=config.LORA_R,
                target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
                lora_alpha=config.LORA_ALPHA,
                lora_dropout=config.LORA_DROPOUT,
                bias="none",
                use_gradient_checkpointing=True,  # Tối ưu memory cho P100
                use_rslora=False,  # Không dùng RSLoRA cho P100
                loftq_config=None,  # Không dùng LoftQ
            )
        
        # Setup model cho training với Unsloth (QUAN TRỌNG!)
        # Không dùng for_training() vì có thể gây conflict với Trainer
        # Unsloth sẽ tự động optimize trong training loop
        
        print("✓ Model đã được tối ưu với Unsloth cho P100!")
        print(f"  - Batch size: {config.BATCH_SIZE}")
        print(f"  - Gradient accumulation: {config.GRADIENT_ACCUMULATION_STEPS}")
        print(f"  - Effective batch size: {config.BATCH_SIZE * config.GRADIENT_ACCUMULATION_STEPS}")
        model.print_trainable_parameters()
        
    except ImportError:
        print("⚠ Unsloth chưa được cài đặt. Chạy với code thông thường...")
        config.USE_UNSLOTH = False  # Fallback to normal

# Load model thông thường (nếu không dùng Unsloth hoặc model không hỗ trợ)
if not config.USE_UNSLOTH or config.MODEL_TYPE not in ["qwen"]:
    print(f"Đang load model: {config.MODEL_NAME} (type: {config.MODEL_TYPE})")
    
    tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME, trust_remote_code=True)
    
    # Xử lý pad_token cho các model khác nhau
    if tokenizer.pad_token is None:
        if hasattr(tokenizer, 'eos_token') and tokenizer.eos_token is not None:
            tokenizer.pad_token = tokenizer.eos_token
        elif hasattr(tokenizer, 'unk_token') and tokenizer.unk_token is not None:
            tokenizer.pad_token = tokenizer.unk_token
        else:
            tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    
    # Load model theo loại
    if config.MODEL_TYPE in ["nllb", "mt5", "helsinki"]:
        # Các model translation thường là Seq2Seq
        # Tối ưu memory: dùng low_cpu_mem_usage và max_memory
        try:
            model = AutoModelForSeq2SeqLM.from_pretrained(
                config.MODEL_NAME,
                trust_remote_code=True,
                torch_dtype=torch.float16,
                device_map="auto",
                low_cpu_mem_usage=True,
                max_memory={0: "13GiB"}  # Giới hạn memory để tránh OOM
            )
        except:
            # Fallback nếu max_memory không hoạt động
            model = AutoModelForSeq2SeqLM.from_pretrained(
                config.MODEL_NAME,
                trust_remote_code=True,
                torch_dtype=torch.float16,
                device_map="auto"
            )
        print("✓ Loaded Seq2Seq model (NLLB/mT5/Helsinki)")
    elif config.MODEL_TYPE == "qwen":
        # Qwen là CausalLM
        from transformers import AutoModelForCausalLM
        model = AutoModelForCausalLM.from_pretrained(
            config.MODEL_NAME,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        print("✓ Loaded CausalLM model (Qwen)")
    else:
        # Thử Seq2Seq trước, nếu không có thì dùng CausalLM
        try:
            model = AutoModelForSeq2SeqLM.from_pretrained(
                config.MODEL_NAME,
                trust_remote_code=True,
                torch_dtype=torch.float16,
                device_map="auto"
            )
            print("✓ Loaded Seq2Seq model (auto-detect)")
        except:
            from transformers import AutoModelForCausalLM
            model = AutoModelForCausalLM.from_pretrained(
                config.MODEL_NAME,
                trust_remote_code=True,
                torch_dtype=torch.float16,
                device_map="auto"
            )
            print("✓ Loaded CausalLM model (auto-detect)")

    # Áp dụng LoRA
    if config.USE_LORA:
        print("Áp dụng LoRA...")
        
        # Đảm bảo model không bị freeze trước khi áp dụng LoRA
        for param in model.parameters():
            param.requires_grad = False  # Freeze tất cả trước, LoRA sẽ unfreeze các modules được target
        
        # Tìm target modules dựa trên model type
        if config.MODEL_TYPE in ["mt5"]:
            # mT5/T5 dùng tên modules đơn giản: "q", "k", "v", "o"
            target_modules = ["q", "k", "v", "o"]
            print(f"✓ Dùng target modules cho mT5: {target_modules}")
        elif config.MODEL_TYPE in ["helsinki"]:
            # Helsinki-NLP (Marian) dùng "q_proj", "k_proj", "v_proj", "o_proj"
            # Nhưng có thể cần tìm động vì tên có thể khác
            try:
                # Tìm tất cả linear layers trong attention
                target_modules = []
                for name, module in model.named_modules():
                    if isinstance(module, torch.nn.Linear):
                        # Tìm các linear layers trong attention
                        if any(x in name.lower() for x in ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'q_lin', 'k_lin', 'v_lin', 'o_lin']):
                            module_name = name.split('.')[-1]
                            if module_name not in target_modules:
                                target_modules.append(module_name)
                
                # Nếu không tìm thấy, dùng default cho Marian
                if not target_modules:
                    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]
                print(f"✓ Tìm thấy target modules cho Helsinki: {target_modules}")
            except:
                # Fallback: dùng default cho Marian
                target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]
                print(f"✓ Dùng default target modules cho Helsinki: {target_modules}")
        else:
            # Default cho các model khác (NLLB, etc.)
            target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]
            print(f"✓ Dùng default target modules: {target_modules}")
        
        # Xác định task type dựa trên model type
        # QUAN TRỌNG: mT5/T5 cần TaskType.SEQ_2_SEQ_LM hoặc không cần task_type
        # Nhưng PEFT có thể tự detect, nên thử không set task_type trước
        if config.MODEL_TYPE in ["nllb", "mt5", "helsinki"]:
            # Với mT5/T5, PEFT sẽ tự detect task type từ model
            # Nhưng để chắc chắn, dùng SEQ_2_SEQ_LM
            task_type = TaskType.SEQ_2_SEQ_LM
        else:
            task_type = TaskType.CAUSAL_LM  # Cho Qwen và các causal models
        
        print(f"✓ Task type: {task_type}, Target modules: {target_modules}")
        
        lora_config = LoraConfig(
            task_type=task_type,
            r=config.LORA_R,
            lora_alpha=config.LORA_ALPHA,
            lora_dropout=config.LORA_DROPOUT,
            target_modules=target_modules,
            bias="none"
        )
        
        try:
            model = get_peft_model(model, lora_config)
            print("✓ LoRA đã được áp dụng")
        except Exception as e:
            print(f"⚠ Lỗi khi áp dụng LoRA với target_modules={target_modules}: {e}")
            print("Thử với target_modules động...")
            # Fallback: tìm tất cả linear layers có chứa q, k, v, o
            import re
            all_modules = []
            for name, module in model.named_modules():
                if isinstance(module, torch.nn.Linear):
                    # Tìm các linear layers trong attention
                    if any(x in name.lower() for x in ['q', 'k', 'v', 'o']):
                        # Lấy tên module (phần cuối)
                        module_name = name.split('.')[-1]
                        if module_name not in all_modules:
                            all_modules.append(module_name)
            
            if all_modules:
                print(f"✓ Tìm thấy {len(all_modules)} modules: {all_modules[:10]}")
                lora_config.target_modules = all_modules
                model = get_peft_model(model, lora_config)
                print("✓ LoRA đã được áp dụng với modules động")
            else:
                # Cuối cùng: dùng all-linear
                print("Thử với target_modules='all-linear'...")
                lora_config = LoraConfig(
                    task_type=task_type,
                    r=config.LORA_R,
                    lora_alpha=config.LORA_ALPHA,
                    lora_dropout=config.LORA_DROPOUT,
                    target_modules="all-linear",
                    bias="none"
                )
                model = get_peft_model(model, lora_config)
                print("✓ LoRA đã được áp dụng với 'all-linear'")
        
        # Đảm bảo model ở training mode
        model.train()
        
        # Đảm bảo LoRA parameters có requires_grad=True
        for name, param in model.named_parameters():
            if "lora" in name.lower():
                param.requires_grad = True
        
        model.print_trainable_parameters()
        
        # Kiểm tra xem có trainable parameters không
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        if trainable_params == 0:
            raise RuntimeError("Không có trainable parameters! LoRA có thể chưa được áp dụng đúng.")
        print(f"✓ Trainable parameters: {trainable_params:,}")
        
        # In ra một số trainable parameter names để debug
        trainable_names = [name for name, param in model.named_parameters() if param.requires_grad]
        print(f"✓ Số lượng trainable modules: {len(trainable_names)}")
        if len(trainable_names) > 0:
            print(f"✓ Ví dụ trainable modules: {trainable_names[:5]}")


Đang tải mô hình...
✓ Detected model type: helsinki
Đang load model: Helsinki-NLP/opus-mt-vi-en (type: helsinki)


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/756k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/289M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/289M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

✓ Loaded Seq2Seq model (NLLB/mT5/Helsinki)
Áp dụng LoRA...
✓ Tìm thấy target modules cho Helsinki: ['k_proj', 'v_proj', 'q_proj']
✓ Task type: TaskType.SEQ_2_SEQ_LM, Target modules: ['k_proj', 'v_proj', 'q_proj']
✓ LoRA đã được áp dụng
trainable params: 884,736 || all params: 73,061,888 || trainable%: 1.2109
✓ Trainable parameters: 884,736
✓ Số lượng trainable modules: 108
✓ Ví dụ trainable modules: ['base_model.model.model.encoder.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.encoder.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.encoder.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.encoder.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.encoder.layers.0.self_attn.q_proj.lora_A.default.weight']


In [7]:
# ============================================================================
# CELL 7: Preprocess Data
# ============================================================================
print("Đang tiền xử lý dữ liệu...")

def preprocess_function(examples):
    """Preprocess function cho dataset - FIXED để loss không = 0
    
    QUAN TRỌNG: Không padding trong preprocessing, để DataCollator làm việc đó.
    Điều này đảm bảo labels được xử lý đúng cách.
    """
    sources = examples["source"]
    targets = examples["target"]
    
    # Format prompt dựa trên model type
    if config.MODEL_TYPE == "mt5":
        lang_map = {"en": "English", "vi": "Vietnamese"}
        if config.DIRECTION == "vi2en":
            prompt_prefix = f"translate {lang_map['en']} to {lang_map['vi']}: "
        else:
            prompt_prefix = f"translate {lang_map['vi']} to {lang_map['en']}: "
        sources = [prompt_prefix + src for src in sources]
    elif config.MODEL_TYPE == "helsinki":
        # Helsinki-NLP không cần prompt, chỉ cần source text
        pass
    else:
        # Các model khác dùng create_translation_prompt
        sources = [create_translation_prompt(text, source_lang, target_lang, config.MODEL_TYPE) 
                   for text in sources]
    
    # Tokenize sources (encoder input) - KHÔNG padding, để DataCollator làm
    model_inputs = tokenizer(
        sources,
        max_length=config.MAX_LENGTH,
        padding=False,  # QUAN TRỌNG: Không padding ở đây
        truncation=True,
        return_tensors=None
    )
    
    # Tokenize targets (labels) - KHÔNG padding, để DataCollator làm
    # QUAN TRỌNG: Dùng text_target để tokenize đúng cách cho Seq2Seq
    labels = tokenizer(
        text_target=targets,
        max_length=config.MAX_LENGTH,
        padding=False,  # QUAN TRỌNG: Không padding ở đây
        truncation=True,
        return_tensors=None
    )
    
    # Labels: thay pad_token_id bằng -100 (ignore trong loss)
    # QUAN TRỌNG: Chỉ thay thế pad_token_id, không padding
    pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0
    labels_input_ids = labels["input_ids"]
    
    # Xử lý labels: thay pad_token_id bằng -100
    processed_labels = []
    for label_seq in labels_input_ids:
        # Chuyển sang list nếu cần
        if not isinstance(label_seq, list):
            label_seq = label_seq.tolist() if hasattr(label_seq, 'tolist') else list(label_seq)
        
        # Thay pad_token_id bằng -100
        processed_label = [l if l != pad_token_id else -100 for l in label_seq]
        processed_labels.append(processed_label)
    
    model_inputs["labels"] = processed_labels
    
    # Debug: Chỉ in một vài lần đầu
    if len(model_inputs["labels"]) <= 3:
        sample_label = model_inputs["labels"][0]
        num_valid_labels = sum(1 for l in sample_label if l != -100)
        if num_valid_labels == 0:
            print(f"⚠ CẢNH BÁO: Sample label toàn là -100!")
        else:
            print(f"✓ Sample label có {num_valid_labels} tokens hợp lệ (tổng {len(sample_label)} tokens)")
    
    return model_inputs

# Tạo datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

val_dataset = val_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=val_dataset.column_names
)

print("Dữ liệu đã sẵn sàng!")


Đang tiền xử lý dữ liệu...


Map:   0%|          | 0/450000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dữ liệu đã sẵn sàng!


In [8]:
# ============================================================================
# CELL 8: Training
# ============================================================================
# Suppress warning về label_names cho PeftModel (không ảnh hưởng đến training)
import warnings
warnings.filterwarnings("ignore", message=".*label_names.*")

print("Bắt đầu training...")

# Kiểm tra model đã được load chưa
if 'model' not in globals() or model is None:
    raise RuntimeError("Model chưa được load! Vui lòng chạy các cell trước đó (CELL 6: Load Model) trước.")
if 'tokenizer' not in globals() or tokenizer is None:
    raise RuntimeError("Tokenizer chưa được load! Vui lòng chạy các cell trước đó (CELL 6: Load Model) trước.")
if 'train_dataset' not in globals() or train_dataset is None:
    raise RuntimeError("Train dataset chưa được load! Vui lòng chạy các cell trước đó (CELL 7: Preprocess Data) trước.")

# Kiểm tra model có trainable parameters không
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
if trainable_params == 0:
    raise RuntimeError("Model không có trainable parameters! Vui lòng kiểm tra lại LoRA configuration.")
print(f"✓ Model có {trainable_params:,} trainable parameters")

# QUAN TRỌNG: Tạo data_collator TRƯỚC khi test forward pass
print("Đang tạo data collator...")
# Data collator - FIXED để đảm bảo labels được xử lý đúng
# QUAN TRỌNG: DataCollatorForSeq2Seq sẽ tự động tạo decoder_input_ids từ labels
# và shift labels để tính loss đúng cách
if config.USE_UNSLOTH:
    # Unsloth có thể cần data collator đặc biệt
    from transformers import DataCollatorForLanguageModeling
    # Thử dùng Seq2Seq collator trước, nếu lỗi thì dùng LanguageModeling
    try:
        data_collator = DataCollatorForSeq2Seq(
            tokenizer=tokenizer,
            padding=True,
            pad_to_multiple_of=8,  # Tối ưu cho Unsloth
            return_tensors="pt"  # Đảm bảo trả về tensors
        )
    except:
        data_collator = DataCollatorForLanguageModeling(
            tokenizer=tokenizer,
            mlm=False,
            pad_to_multiple_of=8,
        )
else:
    # FIXED: DataCollatorForSeq2Seq sẽ tự động:
    # 1. Padding sequences
    # 2. Tạo decoder_input_ids từ labels (thêm decoder_start_token_id)
    # 3. Shift labels để tính loss đúng (labels[i] = decoder_input_ids[i+1])
    
    data_collator = DataCollatorForSeq2Seq(
        tokenizer=tokenizer,
        padding=True,  # Padding sequences
        return_tensors="pt",  # Trả về PyTorch tensors
        model=model  # Truyền model để collator biết decoder_start_token_id
    )
print("✓ Data collator đã được tạo")

# Test forward pass để đảm bảo loss có gradient và KHÔNG = 0
print("Đang test forward pass với gradient...")
try:
    # Lấy một batch nhỏ từ train dataset
    from torch.utils.data import DataLoader
    sample_loader = DataLoader(train_dataset, batch_size=2)
    sample_batch = next(iter(sample_loader))
    
    # QUAN TRỌNG: Dùng data collator để format đúng batch
    # DataCollatorForSeq2Seq sẽ tự động:
    # 1. Padding sequences
    # 2. Tạo decoder_input_ids từ labels (thêm decoder_start_token_id)
    # 3. Shift labels để tính loss đúng (labels[i] = decoder_input_ids[i+1])
    
    # Convert batch sang list of dicts nếu cần
    if isinstance(sample_batch, dict):
        # Nếu đã là dict (từ DataLoader), cần convert sang list of dicts
        batch_size = len(sample_batch[list(sample_batch.keys())[0]])
        sample_batch = [{k: v[i] for k, v in sample_batch.items()} for i in range(batch_size)]
    
    # Dùng data collator để format đúng
    sample_batch = data_collator(sample_batch)
    
    # Chuyển sang device
    device = next(model.parameters()).device
    sample_batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v 
                    for k, v in sample_batch.items()}
    
    # Debug: Kiểm tra batch có đầy đủ keys không
    print(f"✓ Batch keys: {list(sample_batch.keys())}")
    
    # Debug: Kiểm tra labels và decoder_input_ids
    if "labels" in sample_batch:
        labels = sample_batch["labels"]
        num_valid_labels = (labels != -100).sum().item()
        print(f"✓ Batch có {num_valid_labels} valid labels (tổng {labels.numel()} tokens)")
        print(f"   Labels shape: {labels.shape}, dtype: {labels.dtype}")
        print(f"   Labels min/max: {labels.min().item()}/{labels.max().item()}")
        if num_valid_labels == 0:
            print("⚠ CẢNH BÁO: Tất cả labels đều là -100! Loss sẽ = 0!")
            print("   Kiểm tra lại preprocessing function và data collator.")
    
    # QUAN TRỌNG: Kiểm tra decoder_input_ids (mT5 cần cái này)
    if "decoder_input_ids" in sample_batch:
        decoder_input_ids = sample_batch["decoder_input_ids"]
        print(f"✓ Có decoder_input_ids: shape {decoder_input_ids.shape}")
        print(f"   Decoder input min/max: {decoder_input_ids.min().item()}/{decoder_input_ids.max().item()}")
        # Kiểm tra decoder_start_token_id (mT5 thường dùng 0 hoặc pad_token_id)
        if decoder_input_ids.shape[0] > 0:
            first_token = decoder_input_ids[0, 0].item()
            print(f"   First decoder token (should be decoder_start_token_id): {first_token}")
    else:
        print("⚠ CẢNH BÁO: Không có decoder_input_ids! DataCollator có thể chưa tạo đúng.")
        print("   Với mT5/T5, decoder_input_ids là BẮT BUỘC!")
    
    # Forward pass VỚI gradient (không dùng torch.no_grad)
    model.train()  # Đảm bảo training mode
    outputs = model(**sample_batch)
    
    # Kiểm tra loss
    if hasattr(outputs, 'loss') and outputs.loss is not None:
        loss = outputs.loss
        loss_value = loss.item()
        print(f"✓ Forward pass thành công, loss: {loss_value:.4f}")
        
        # QUAN TRỌNG: Kiểm tra loss có = 0 không
        if loss_value == 0.0:
            print("⚠⚠⚠ LOSS = 0! Đây là vấn đề nghiêm trọng!")
            print("   Nguyên nhân có thể:")
            print("   1. Labels toàn là -100 (đã được ignore)")
            print("   2. Model không tính loss đúng cách")
            print("   3. TaskType hoặc LoRA config sai")
            print("   Đang kiểm tra...")
            
            # Kiểm tra labels trong batch
            if "labels" in sample_batch:
                labels_check = sample_batch["labels"]
                print(f"   - Labels shape: {labels_check.shape}")
                print(f"   - Labels min/max: {labels_check.min().item()}/{labels_check.max().item()}")
                print(f"   - Số labels = -100: {(labels_check == -100).sum().item()}")
                print(f"   - Số labels != -100: {(labels_check != -100).sum().item()}")
            
            # Kiểm tra logits
            if hasattr(outputs, 'logits'):
                logits = outputs.logits
                print(f"   - Logits shape: {logits.shape}")
                print(f"   - Logits min/max: {logits.min().item():.4f}/{logits.max().item():.4f}")
                
                # Kiểm tra xem logits có đúng shape không (phải match với labels)
                if "labels" in sample_batch:
                    labels_shape = sample_batch["labels"].shape
                    # Logits shape: [batch_size, seq_len, vocab_size]
                    # Labels shape: [batch_size, seq_len]
                    if len(logits.shape) == 3 and len(labels_shape) == 2:
                        if logits.shape[0] == labels_shape[0] and logits.shape[1] == labels_shape[1]:
                            print(f"   - ✓ Logits và labels shape khớp")
                        else:
                            print(f"   - ⚠ Logits và labels shape KHÔNG khớp!")
                            print(f"     Logits: {logits.shape}, Labels: {labels_shape}")
                
                # Kiểm tra xem có NaN hoặc Inf không
                if torch.isnan(logits).any():
                    print(f"   - ⚠ Logits có NaN!")
                if torch.isinf(logits).any():
                    print(f"   - ⚠ Logits có Inf!")
                
                # Tính loss thủ công để debug
                if "labels" in sample_batch:
                    from torch.nn import CrossEntropyLoss
                    labels_for_loss = sample_batch["labels"]
                    # Reshape logits và labels để tính loss
                    shift_logits = logits[..., :-1, :].contiguous()
                    shift_labels = labels_for_loss[..., 1:].contiguous()
                    loss_fct = CrossEntropyLoss(ignore_index=-100)
                    manual_loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
                    print(f"   - Loss tính thủ công: {manual_loss.item():.4f}")
                    if manual_loss.item() == 0.0:
                        print(f"   - ⚠⚠⚠ Loss tính thủ công cũng = 0!")
                        print(f"     Có thể do labels toàn là -100 sau khi shift")
                        print(f"     Shift labels valid tokens: {(shift_labels != -100).sum().item()}")
        
        # Kiểm tra loss có gradient không
        if loss.requires_grad:
            print(f"✓ Loss có requires_grad=True")
            # Test backward nhỏ
            loss.backward()
            print(f"✓ Backward pass thành công")
            # Clear gradients
            model.zero_grad()
        else:
            print("⚠ LOSS KHÔNG CÓ GRADIENT! Đây là vấn đề!")
            print("Có thể do gradient checkpointing xung đột với LoRA")
            print("Sẽ tắt gradient checkpointing và thử lại...")
            config.GRADIENT_CHECKPOINTING = False
    else:
        print("⚠ Forward pass không trả về loss, có thể cần kiểm tra lại")
        print(f"   Outputs keys: {outputs.keys() if hasattr(outputs, 'keys') else 'N/A'}")
    
except Exception as e:
    print(f"⚠ Lỗi khi test forward pass: {e}")
    import traceback
    traceback.print_exc()
    print("Tiếp tục training...")

# Điều chỉnh batch size cho Unsloth (Unsloth có thể tự động điều chỉnh)
train_batch_size = config.BATCH_SIZE
eval_batch_size = config.BATCH_SIZE * 2

# Unsloth có thể cần batch size khác
if config.USE_UNSLOTH:
    # Unsloth thường dùng batch size lớn hơn, nhưng giữ nguyên để tránh OOM
    pass

training_args = TrainingArguments(
    output_dir=config.OUTPUT_DIR,
    num_train_epochs=config.NUM_EPOCHS,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    gradient_accumulation_steps=config.GRADIENT_ACCUMULATION_STEPS,
    learning_rate=config.LEARNING_RATE,
    warmup_steps=50,  # Giảm warmup steps (từ 100 xuống 50)
    weight_decay=0.01,
    logging_steps=getattr(config, 'LOGGING_STEPS', 100),  # Dùng config hoặc default 100
    eval_steps=getattr(config, 'EVAL_STEPS', 500) if config.EVAL_STRATEGY != "no" else None,
    save_steps=getattr(config, 'SAVE_STEPS', 1000),  # Dùng config hoặc default 1000
    eval_strategy=getattr(config, 'EVAL_STRATEGY', "no"),  # Tắt eval để nhanh hơn
    save_strategy="steps",
    load_best_model_at_end=False if config.EVAL_STRATEGY == "no" else True,  # Tắt nếu không eval
    fp16=True,
    gradient_checkpointing=config.GRADIENT_CHECKPOINTING,
    dataloader_pin_memory=False,  # Tắt pin_memory để tiết kiệm memory
    dataloader_num_workers=2,  # Giảm workers (từ 4 xuống 2) để nhanh hơn
    report_to="none",
    remove_unused_columns=False,  # Giữ tất cả columns để tránh lỗi
    # Tối ưu thêm
    save_total_limit=1,  # Chỉ giữ 1 checkpoint (tiết kiệm disk và thời gian)
    prediction_loss_only=True,  # Chỉ tính loss, không tính metrics (nhanh hơn)
)

# Đảm bảo model ở training mode trước khi tạo Trainer
model.train()
for param in model.parameters():
    if param.requires_grad:
        param.requires_grad = True  # Đảm bảo trainable params có gradient

# Chỉ thêm eval_dataset nếu có evaluation
trainer_kwargs = {
    "model": model,
    "args": training_args,
    "train_dataset": train_dataset,
    "data_collator": data_collator,
    "processing_class": tokenizer
}

# Chỉ thêm eval_dataset nếu eval_strategy không phải "no"
if getattr(config, 'EVAL_STRATEGY', 'no') != "no":
    trainer_kwargs["eval_dataset"] = val_dataset

trainer = Trainer(**trainer_kwargs)

# Kiểm tra lại model sau khi tạo Trainer
print(f"✓ Model training mode: {model.training}")
print(f"✓ Trainable params: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# Training với Unsloth optimization
print(f"Training với {config.NUM_EPOCHS} epoch(s), {config.MAX_SAMPLES} samples")
print(f"Batch size: {config.BATCH_SIZE}, Gradient accumulation: {config.GRADIENT_ACCUMULATION_STEPS}")
print(f"Effective batch size: {config.BATCH_SIZE * config.GRADIENT_ACCUMULATION_STEPS}")
print(f"Max length: {config.MAX_LENGTH}")
print(f"Eval strategy: {getattr(config, 'EVAL_STRATEGY', 'no')}")

# Tính toán số steps ước tính
total_steps = len(train_dataset) // (config.BATCH_SIZE * config.GRADIENT_ACCUMULATION_STEPS) * config.NUM_EPOCHS
print(f"Estimated total steps: ~{total_steps}")
print("Bắt đầu training...\n")

trainer.train()

# Save model - Unsloth compatible
if config.USE_UNSLOTH:
    # Unsloth có cách save riêng
    try:
        model.save_pretrained(config.OUTPUT_DIR)
        tokenizer.save_pretrained(config.OUTPUT_DIR)
        print("✓ Model đã được lưu (Unsloth format)")
    except Exception as e:
        print(f"⚠ Lỗi khi save với Unsloth format: {e}")
        print("Thử save với format thông thường...")
        trainer.save_model()
        tokenizer.save_pretrained(config.OUTPUT_DIR)
else:
    trainer.save_model()
    tokenizer.save_pretrained(config.OUTPUT_DIR)

print("Training hoàn thành!")

Traceback (most recent call last):
  File "/tmp/ipykernel_20/2670660596.py", line 66, in <cell line: 0>
    sample_batch = next(iter(sample_loader))
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 708, in __next__
    data = self._next_data()
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 764, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 55, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/collate.py", line 398, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/u

Bắt đầu training...
✓ Model có 884,736 trainable parameters
Đang tạo data collator...
✓ Data collator đã được tạo
Đang test forward pass với gradient...
⚠ Lỗi khi test forward pass: each element in list of batch should be of equal size
Tiếp tục training...
✓ Model training mode: True
✓ Trainable params: 884,736
Training với 1 epoch(s), 500000 samples
Batch size: 8, Gradient accumulation: 4
Effective batch size: 32
Max length: 128
Eval strategy: no
Estimated total steps: ~14062
Bắt đầu training...



Step,Training Loss
100,3.748300
200,3.648000
300,3.619500
400,3.597000
500,3.556600
600,3.543300
700,3.523800
800,3.468800
900,3.468200
1000,3.451600


Training hoàn thành!


In [9]:
# ============================================================================
# CELL 9: Evaluation
# ============================================================================
# Suppress warning về label_names cho PeftModel (không ảnh hưởng đến evaluation)
import warnings
warnings.filterwarnings("ignore", message=".*label_names.*")

print("Đang đánh giá mô hình...")

# Load test sources và targets
test_sources = test_df["source"].tolist()
test_targets = test_df["target"].tolist()

# Generate translations
model.eval()
predictions = []

with torch.no_grad():
    for i in tqdm(range(0, len(test_sources), config.BATCH_SIZE)):
        batch_sources = test_sources[i:i+config.BATCH_SIZE]
        
        # mT5 cần prompt format
        if config.MODEL_TYPE == "mt5":
            lang_map = {"en": "English", "vi": "Vietnamese"}
            if config.DIRECTION == "vi2en":
                prompt_prefix = f"translate {lang_map['en']} to {lang_map['vi']}: "
            else:
                prompt_prefix = f"translate {lang_map['vi']} to {lang_map['en']}: "
            prompts = [prompt_prefix + src for src in batch_sources]
        else:
            prompts = [create_translation_prompt(text, source_lang, target_lang, config.MODEL_TYPE) 
                       for text in batch_sources]
        
        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=config.MAX_LENGTH
        )
        
        if torch.cuda.is_available():
            inputs = {k: v.to(model.device) for k, v in inputs.items()}
        
        # Tối ưu generation: giảm num_beams để nhanh hơn
        outputs = model.generate(
            **inputs,
            max_length=config.MAX_LENGTH,
            num_beams=2,  # Giảm từ 4 xuống 2 để nhanh hơn (có thể dùng greedy: num_beams=1)
            early_stopping=True,
            do_sample=False,  # Không sample để nhanh hơn
        )
        
        batch_preds = tokenizer.batch_decode(
            outputs,
            skip_special_tokens=True
        )
        predictions.extend(batch_preds)

# Lưu predictions
with open(config.PREDICTIONS_FILE, 'w', encoding='utf-8') as f:
    for pred in predictions:
        f.write(pred + '\n')

# Tính BLEU
refs = [[ref] for ref in test_targets]
bleu = sacrebleu.corpus_bleu(predictions, refs, tokenize="flores200")

results = {
    "bleu": bleu.score,
    "bleu_precision": bleu.precisions,
    "num_samples": len(predictions)
}

# Lưu kết quả
with open(config.RESULTS_FILE, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"BLEU Score: {bleu.score:.4f}")
print(f"Kết quả đã lưu vào: {config.RESULTS_FILE}")

Đang đánh giá mô hình...


100%|██████████| 375/375 [05:46<00:00,  1.08it/s]


BLEU Score: 31.9337
Kết quả đã lưu vào: /kaggle/working/evaluation_results.json
